In [1]:
import numpy as np
import networkx as nx
import algorithms
import evaluate

Scheme works as follows:
1. Coarsen input graph with some __edge merging algorithm__
2. Partition coarsest graph with some __partitioning algorithm__
3. Project partition of coarsest graph to partition of input graph

# Test: SBM

In [45]:
sizes = np.array([50, 50, 50, 50])
p = np.array([[.8, .1, .1, .1],
              [.1, .8, .1, .1],
              [.1, .1, .8, .1],
              [.1, .1 ,.1 ,.8]])
G = nx.stochastic_block_model(sizes=sizes, p=p)
A = nx.to_numpy_array(G)

In [26]:
sizes = np.array([150, 50])
p = np.array([[.4, .1],
              [.1, .4]])
G = nx.stochastic_block_model(sizes=sizes, p=p)
A = nx.to_numpy_array(G)

In [46]:
def my_temperature_merging(A, node_weights):
    return algorithms.temperature_merging(A=A, node_weights=node_weights, temperature=.5)

In [65]:
merge_names = ['Temperature merging', 
               'Random matching', 
               'Heavy edge matching', 
               'Max-cut matching']

x_blocks = np.zeros(A.shape[0])
x_blocks[:sizes[0]] = 1
block_cut = evaluate.wcut(x=x_blocks, A=A)
block_fractions = evaluate.evaluate_SBM_partition(x=x_blocks, sizes=sizes)

x_rnd = np.random.binomial(n=1, p=.5, size=A.shape[0])
random_cut = evaluate.wcut(x=x_rnd, A=A)
random_fractions = evaluate.evaluate_SBM_partition(x=x_rnd, sizes=sizes)

x_uncoarsened = algorithms.compute_spectral_wcut(A)
uncoarsened_cut = evaluate.wcut(x=x_uncoarsened, A=A)
uncoarsend_fractions = evaluate.evaluate_SBM_partition(x=x_uncoarsened, sizes=sizes)

print(f'Block cut: {block_cut}, {block_fractions}')
print(f'Random cut: {random_cut}, {random_fractions}')
print(f'Uncoarsened cut: {uncoarsened_cut}, {uncoarsend_fractions}')  

for i, merge_fn in enumerate([my_temperature_merging,
                              algorithms.random_matching_merging,
                              algorithms.heavy_edge_merging, 
                              algorithms.max_cut_merging]):    
    print(f'Merging strategy: {merge_names[i]}')
    find_cuts = algorithms.FindCuts(A=A, merge_fn=merge_fn, partition_fn=algorithms.compute_spectral_wcut)
    for N_max in [50]:
        x = find_cuts(N_max=N_max)
        cut_val = evaluate.wcut(x=x, A=A)
        fractions = evaluate.evaluate_SBM_partition(x=x, sizes=sizes)
        print(f'N_max={N_max}: fractions {fractions} and cut value {cut_val}')

Block cut: 20.053333333333335, [1.0, 0.0, 0.0, 0.0]
Random cut: 54.175438596491226, [0.46, 0.5, 0.48, 0.66]
Uncoarsened cut: 19.78666666666667, [1.0, 0.0, 1.0, 1.0]
Merging strategy: Temperature merging
N_max=50: fractions [0.52, 0.98, 0.98, 0.96] and cut value 42.8156146179402
Merging strategy: Random matching
N_max=50: fractions [0.12, 0.72, 0.6, 0.2] and cut value 44.91525423728814
Merging strategy: Heavy edge matching
N_max=50: fractions [0.5, 0.9, 0.14, 0.82] and cut value 39.9131872674659
Merging strategy: Max-cut matching
N_max=50: fractions [0.64, 0.92, 0.16, 0.96] and cut value 37.7431026684758


# Iterative algorithm to find coarser cuts

In [117]:
class HierarchicalCuts(object):
    def __init__(self, A, get_bipartition, a):
        self.A = A
        self.get_bipartition = get_bipartition
        self.a = a
        self.list_of_cuts = []
    
    def __call__(self, threshold):
        print(f'Shape of A: {A.shape}, number of edges {A.sum()/2}')
        x = get_bipartition(A)
        self.list_of_cuts.append(x) 
        print(f'Added initial cut with lenghts {[x.sum(), (1-x).sum()]} with ratio {self.evaluate_cut(x=x, A=A)}')
        for y in [0, 1]:
            self.get_cuts(A_sub=A[np.ix_(x==y, x==y)], node_idxs=np.where(x==y)[0], threshold=threshold)
    
    def get_cuts(self, A_sub, node_idxs, threshold):
        print(f'Shape of A: {A_sub.shape}, number of edges {A_sub.sum()/2}')
        x_sub = get_bipartition(A_sub)
        # Test if cut through subgraph is a meaningful partition. If no, stop. If yes, add both cuts
        ratio = self.evaluate_cut(x=x_sub, A=A_sub)
        print(f'ratio: {ratio}, sizes: {[x_sub.sum(), (1-x_sub).sum()]}')
        print(ratio  < threshold and min(x_sub.sum(), (1-x_sub).sum()) > self.a)
        if ratio  < threshold and min(x_sub.sum(), (1-x_sub).sum()) > self.a:
            print(f'Added two new cuts with lenghts {[x_sub.sum(), (1-x_sub).sum()]} and ratio {ratio}')
            for y in [0, 1]:
                x = np.zeros(self.A.shape[0])
                x[node_idxs[x_sub==y]] = 1
                self.list_of_cuts.append(x)
                # Recursively call function on this subset
                self.get_cuts(A_sub=A_sub[np.ix_(x_sub==y, x_sub==y)], 
                              node_idxs=node_idxs[np.where(x_sub==y)[0]],
                              threshold=threshold)
        print('Reached leaf')
        return
    
    def evaluate_cut(self, x, A):
        x_rnd = np.random.binomial(n=1, p=.5, size=A.shape[0])
        val = evaluate.wcut(x=x, A=A)
        val_rnd = evaluate.wcut(x=x_rnd, A=A)        
        return val / val_rnd

# Example

In [134]:
sizes = np.array([200, 100, 100])
p = np.array([[.6, .1, .1],
              [.1, .6, .1],
              [.1, .1, .6]])
G = nx.stochastic_block_model(sizes=sizes, p=p)
A = nx.to_numpy_array(G)

In [145]:
# Need to rewrite FindCuts as a function instead of a class
def get_bipartition(A):
    find_cuts = algorithms.FindCuts(A=A, merge_fn=algorithms.random_matching_merging, 
                                    partition_fn=algorithms.compute_spectral_wcut)
    return find_cuts(N_max=100, verbose=False)

In [148]:
hierarchical_cuts = HierarchicalCuts(A=A, get_bipartition=get_bipartition, a=20)
hierarchical_cuts(threshold=.9)
[evaluate.evaluate_SBM_partition(x=x, sizes=sizes) for x in hierarchical_cuts.list_of_cuts]

Shape of A: (400, 400), number of edges 22846.0
Added initial cut with lenghts [103.0, 297.0] with ratio 0.7640474728793062
Shape of A: (297, 297), number of edges 14601.0
ratio: 0.49034430469399093, sizes: [43.0, 254.0]
True
Added two new cuts with lenghts [43.0, 254.0] and ratio 0.49034430469399093
Shape of A: (254, 254), number of edges 12514.0
ratio: 0.34875540605478145, sizes: [250.0, 4.0]
False
Reached leaf
Shape of A: (43, 43), number of edges 315.0
ratio: 0.3105590062111801, sizes: [21.0, 22.0]
True
Added two new cuts with lenghts [21.0, 22.0] and ratio 0.3105590062111801
Shape of A: (22, 22), number of edges 136.0
ratio: 0.8870967741935486, sizes: [4.0, 18.0]
False
Reached leaf
Shape of A: (21, 21), number of edges 129.0
ratio: 0.7205882352941178, sizes: [4.0, 17.0]
False
Reached leaf
Reached leaf
Reached leaf
Shape of A: (103, 103), number of edges 1600.0
ratio: 0.3856706578118184, sizes: [98.0, 5.0]
False
Reached leaf


[[0.065, 0.52, 0.38],
 [0.935, 0.27, 0.4],
 [0.0, 0.21, 0.22],
 [0.0, 0.0, 0.22],
 [0.0, 0.21, 0.0]]